**Implementation**

**Representation of text / Preprocessing:**
I have started by using a multinomial naïve bayes classifier, this means the frequency of occurrence of each word will be recorded instead of a Boolean expression.
 I have also decided to use all words instead of top 1000 words so that some informative words are not missed out. 
For the multinomial version, I created 4 dictionaries based on the corresponding class. Each dictionary has words that occurred as key and frequency of occurrence as value. 
For the complement version, I created one dictionary of dictionary where each word is a key and values are another dictionary that has 4 keys corresponding to 4 classes and the value is the count  of this word occurring in document of each type.
It was found that removing common words had no significant effect on performance, therefore we did not remove the common words. 

**Using logarithm:**
To handle the underflow problem (multiplying many small floating numbers and receive 0), I have decided to apply a simple log-transformation to the probabilities and the final probability will be the sum of all log-probabilities. The final predicted value will be the class with highest log-probability.The log transform is also necessary later on when I implement the complement version of naïve bayes classifier.

**Laplacian smoothing:**
There will be words that are not being recorded in the frequency dictionary (i.e. with a frequency of 0). Since we cannot log-transform 0 and having 0 multiplied by other probabilities will leave everything as 0. I decided to add 1 to all word frequencies so that we have a minimum frequency of 1 and there will be no 0 probabilities. 

**Method extension**

*TF-IDF Document frequency normalization:*
It is obvious that if a word appears in every text, it would be useless for prediction. Whereas high informative words only appear in small number of texts. A TF-IDF document frequency normalization was used such all word frequency has been multiplied by the IDF value of that word. Some rare words would have higher IDF value and common words have lower IDF value. This improvement is aimed to increase the weight of those rarely occurred words so that we do not miss any important information.

*Complement Naïve bayes:*
The training dataset we received is heavily imbalanced. There are 2144 rows of class e, 1602 rows of class b and only around 100 rows for class a and class v. From “Tackling the Poor Assumptions of Naive Bayes Text Classifiers” We know that a complement naïve bayes would perform better than a multinomial naïve bayes on some imbalanced dataset. Therefore, we have attempted to implement a complement version of naïve bayes classifier. 

**Evaluation**
I have split the training data into training and validation set. The size of training set is 70% of all data and 30% of data are used for testing. 
For the multinomial version: We obtained an accuracy of 0.97775 when the entire dataset was used and 0.94 on the validation set. We obtained an accuracy of 0.9533 on Kaggle.
For TF-IDF improved version, we obtained an accuracy of 0.99025 when the entire dataset was used and 0.944 on the validation set. We obtained an accuracy of 0.94 on Kaggle.
For the complement version, We obtained an accuracy of 0.99225 when the entire dataset was used and 0.948 on the validation set. We obtained an accuracy of 0.96 on Kaggle, this is our best score. 
Lastly, for the complement + TF-IDF version we obtained an accuracy as high as 0.9975 when the entire dataset was used and 0.954 on the validation set. However, we only obtained 0.9566 on Kaggle, which is a very minor improvement from the multinomial version. 

**Conclusion**
It is clear that any implementation/extension of naïve bayes classifier preforms far better than the ZeroR model (predicting everything as the majority class) which has an accuracy of 51.3% on Kaggle.  
All of the extensions had better performance than the standard naïve bayes classifier which had an accuracy of around 92%. 
I found that the complement version had a significant improvement than the multinomial version, this is not a surprise because we did have very imbalanced class. 
After also using TF-IDF (Document frequency normalization) on the complement version, the performance on the validation set was increased very slightly. The effect of Document frequency normalization was rather insignificant on the unseen dataset, but it does have a significant effect on the training dataset. 
Overall, the complement + TF-IDF version did have a noticeable higher accuracy than standard naïve bayes model and a much higher accuracy than the majority model. However, the effect of TF-IDF was not obvious, it is possible that this method caused some overfitting. 
Our best score was 0.96 on Kaggle, from the complement class version.


In [ ]:
import numpy as np # linear algebra
import csv 
import math

In [ ]:
# Import files
X = []
y = []
X_test = []
with open("/kaggle/input/naive-bayes/trg.csv", 'r') as file:
    reader = csv.reader(file)
    for data in reader:
        y.append(data[1])
        X.append(data[2])
X = X[1:]
y = y[1:]

with open("/kaggle/input/naive-bayes/tst.csv", 'r') as file:
    reader = csv.reader(file)
    for data in reader:
        X_test.append(data[1])
X_test = X_test[1:]

In [ ]:
# TF-IDF improved Version
def train(X,y):
    # Obtain posterior probability with dictionaries by class
    dict_a = {}
    dict_b = {}
    dict_v = {}
    dict_e = {}
    
    # Obtain number of each class
    count_a = 0
    count_b = 0
    count_v = 0
    count_e = 0
    unique = {}

    # Obtain number of words in each class
    count_dict = {"A":0, "B":0, "E":0, "V":0}

    for i in range(len(X)):
        word_li = X[i].split()
        if y[i] == "A":
            count_a += 1
            for word in word_li:
                count_dict["A"] += 1
                if word not in dict_a:
                    dict_a[word] = 1
                else:
                    dict_a[word]+=1
                if word not in unique:
                    unique[word] = np.zeros((4000,), dtype=int)
                    unique[word][i] = 1
                else:
                    if unique[word][i] == 0:
                        unique[word][i] = 1

        elif y[i] == "B":
            count_b += 1
            for word in word_li:
                count_dict["B"] += 1
                if word not in dict_b:
                    dict_b[word] = 1
                else:
                    dict_b[word] +=1
                if word not in unique:
                    unique[word] = np.zeros((4000,), dtype=int)
                    unique[word][i] = 1
                else:
                    if unique[word][i] == 0:
                        unique[word][i] = 1

        elif y[i] == "E":
            count_e += 1
            for word in word_li:
                count_dict["E"] += 1
                if word not in dict_e:
                    dict_e[word] = 1
                else:
                    dict_e[word] += 1 
                if word not in unique:
                    unique[word] = np.zeros((4000,), dtype=int)
                    unique[word][i] = 1
                else:
                    if unique[word][i] == 0:
                        unique[word][i] = 1

        elif y[i] == "V":
            count_v += 1
            for word in word_li:
                count_dict["V"] += 1
                if word not in dict_v:
                    dict_v[word] = 1
                else:
                    dict_v[word]+=1 
                if word not in unique:
                    unique[word] = np.zeros((4000,), dtype=int)
                    unique[word][i] = 1
                else:
                    if unique[word][i] == 0:
                        unique[word][i] = 1
    # Obtain Prior Probability of Classes
    prior_dict = {}
    prior_dict["A"] = math.log(count_a / len(X))
    prior_dict["B"] = math.log(count_b / len(X))
    prior_dict["V"] = math.log(count_v / len(X))
    prior_dict["E"] = math.log(count_e / len(X))
    
    return prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict

def predict(X, prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict):
    # Predicting outcomes 
    outcome = [[] for x in range(len(X))]
    target = ["A", "B", "E", "V"]
    for i in range(len(X)):
        P_AX = prior_dict["A"]
        P_BX = prior_dict["B"]
        P_EX = prior_dict["E"]
        P_VX = prior_dict["V"]
        unseen = 0
        for word in X[i].split():
            if word not in unique:
                unseen += 1
            if word in dict_a:
                P_AX = P_AX + math.log((dict_a[word]*math.log(4000/unique[word].sum())+1) / (count_dict["A"]+ len(unique) + unseen))
            else:
                P_AX = P_AX + math.log(1/(count_dict["A"] + len(unique)+ unseen))

            if word in dict_b:
                P_BX = P_BX + math.log((dict_b[word]*math.log(4000/unique[word].sum())+1) / (count_dict["B"]+ len(unique) + unseen))
            else:
                P_BX = P_BX + math.log(1/(count_dict["B"] + len(unique)+ unseen))

            if word in dict_e:
                P_EX = P_EX + math.log((dict_e[word]*math.log(4000/unique[word].sum())+1) / (count_dict["E"]+ len(unique) + unseen))
            else:
                P_EX = P_EX + math.log(1/(count_dict["E"] + len(unique) + unseen))

            if word in dict_v:
                P_VX = P_VX + math.log((dict_v[word]*math.log(4000/unique[word].sum())+1)/(count_dict["V"]+ len(unique) + unseen))
            else:
                P_VX = P_VX + math.log(1/(count_dict["V"] + len(unique) + unseen))

        probs = [P_AX, P_BX, P_EX, P_VX]
        outcome[i] = target[probs.index(max(probs))]
    return outcome

def evaluate(result, y):
    score = len(y)
    for i in range(len(y)):
        if result[i] != y[i]:
            score -= 1
    return score/len(y)

# Evaluation
ind = round(len(X)*0.7)
X_train = X[:ind]
X_val = X[ind:]
y_train = y[:ind]
y_val = y[ind:]

prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict = train(X_train, y_train)
result = predict(X_val, prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict)
evaluate(result, y_val)

In [ ]:
# Multinomial version
def train(X,y):
    # Obtain posterior probability
    dict_a = {}
    dict_b = {}
    dict_v = {}
    dict_e = {}
    
    # Obtain number of each class
    count_a = 0
    count_b = 0
    count_v = 0
    count_e = 0
    unique = {}

    # Obtain number of words in each class
    count_dict = {"A":0, "B":0, "E":0, "V":0}

    for i in range(len(X)):
        word_li = X[i].split()
        if y[i] == "A":
            count_a += 1
            for word in word_li:
                count_dict["A"] += 1
                if word not in dict_a:
                    dict_a[word] = 1
                else:
                    dict_a[word]+=1
                if word not in unique:
                    unique[word] = 1
                else:
                    unique[word] += 1

        elif y[i] == "B":
            count_b += 1
            for word in word_li:
                count_dict["B"] += 1
                if word not in dict_b:
                    dict_b[word] = 1
                else:
                    dict_b[word] +=1
                if word not in unique:
                    unique[word] = 1
                else:
                    unique[word] += 1

        elif y[i] == "E":
            count_e += 1
            for word in word_li:
                count_dict["E"] += 1
                if word not in dict_e:
                    dict_e[word] = 1
                else:
                    dict_e[word] += 1 
                if word not in unique:
                    unique[word] = 1
                else:
                    unique[word] += 1

        elif y[i] == "V":
            count_v += 1
            for word in word_li:
                count_dict["V"] += 1
                if word not in dict_v:
                    dict_v[word] = 1
                else:
                    dict_v[word]+=1 
                if word not in unique:
                    unique[word] = 1
                else:
                    unique[word] += 1
    # Obtain Prior Probability of Classes
    prior_dict = {}
    prior_dict["A"] = math.log(count_a / len(X))
    prior_dict["B"] = math.log(count_b / len(X))
    prior_dict["V"] = math.log(count_v / len(X))
    prior_dict["E"] = math.log(count_e / len(X))
    
    return prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict

def predict(X, prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict):
    # Predicting outcomes 
    outcome = [[] for x in range(len(X))]
    target = ["A", "B", "E", "V"]
    for i in range(len(X)):
        P_AX = prior_dict["A"]
        P_BX = prior_dict["B"]
        P_EX = prior_dict["E"]
        P_VX = prior_dict["V"]
        unseen = 0
        for word in X[i].split():
            if word not in unique:
                unseen += 1
            if word in dict_a:
                P_AX = P_AX + math.log((dict_a[word]+1) / (count_dict["A"]+ len(unique) + unseen))
            else:
                P_AX = P_AX + math.log(1/(count_dict["A"] + len(unique)+ unseen))

            if word in dict_b:
                P_BX = P_BX + math.log((dict_b[word]+1) / (count_dict["B"]+ len(unique) + unseen))
            else:
                P_BX = P_BX + math.log(1/(count_dict["B"] + len(unique)+ unseen))

            if word in dict_e:
                P_EX = P_EX + math.log((dict_e[word]+1)/(count_dict["E"]+ len(unique) + unseen))
            else:
                P_EX = P_EX + math.log(1/(count_dict["E"] + len(unique) + unseen))

            if word in dict_v:
                P_VX = P_VX + math.log((dict_v[word]+1)/(count_dict["V"]+ len(unique) + unseen))
            else:
                P_VX = P_VX + math.log(1/(count_dict["V"] + len(unique) + unseen))

        probs = [P_AX, P_BX, P_EX, P_VX]
        outcome[i] = target[probs.index(max(probs))]
    return outcome

def evaluate(result, y):
    score = len(y)
    for i in range(len(y)):
        if result[i] != y[i]:
            score -= 1
    return score/len(y)

# Evaluate
ind = round(len(X)*0.7)
X_train = X[:ind]
X_val = X[ind:]
y_train = y[:ind]
y_val = y[ind:]

prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict = train(X_train, y_train)
result = predict(X_val, prior_dict, dict_a, dict_b, dict_v, dict_e, unique, count_dict)
evaluate(result, y_val)

In [ ]:
# Complement class version
def train(X,y):
    
    # Obtain number of each class
    count_a = 0
    count_b = 0
    count_v = 0
    count_e = 0
    unique = {}
    word_dict = {}
    # Obtain number of words in each class
    count_dict = {"A":0, "B":0, "E":0, "V":0}

    for i in range(len(X)):
        word_li = X[i].split()
        for word in word_li:
            if word not in word_dict:
                word_dict[word]={"A":0, "B":0, "V":0, "E":0}
            if y[i] == "A":
                word_dict[word]["A"] += 1
                count_dict["A"] += 1
            elif y[i] == "B":
                word_dict[word]["B"] += 1
                count_dict["B"] += 1
            elif y[i] == "E":
                word_dict[word]["E"] += 1
                count_dict["E"] += 1
            elif y[i] == "V":
                word_dict[word]["V"] += 1
                count_dict["V"] += 1
                
    # Obtain Prior Probability of Classes
    for i in range(len(X)):
        if y[i] == "A":
            count_a += 1
        elif y[i] == "B":
            count_b += 1
        elif y[i] == "E":
            count_e += 1  
        elif y[i] == "V":
            count_v += 1
            
    prior_dict = {}
    prior_dict["A"] = math.log(count_a / len(X))
    prior_dict["B"] = math.log(count_b / len(X))
    prior_dict["V"] = math.log(count_v / len(X))
    prior_dict["E"] = math.log(count_e / len(X))            
    
    return prior_dict, word_dict, count_dict

def predict(X, prior_dict, word_dict, count_dict):
    # Predicting outcomes 
    outcome = [[] for x in range(len(X))]
    target = ["A", "B", "E", "V"]
    for i in range(len(X)):
        unseen = 0
        P_AX = prior_dict["A"]
        P_BX = prior_dict["B"]
        P_EX = prior_dict["E"]
        P_VX = prior_dict["V"]
        for word in X[i].split():
            if word not in word_dict:
                P_AX = P_AX - math.log(1/(count_dict["B"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_BX = P_BX - math.log(1/(count_dict["A"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_VX = P_VX - math.log(1/(count_dict["A"]+count_dict["B"]+count_dict["E"]+len(word_dict)))
                P_EX = P_EX - math.log(1/(count_dict["A"]+count_dict["B"]+count_dict["V"]+len(word_dict)))
            else:
                P_AX = P_AX - math.log((word_dict[word]["B"]+word_dict[word]["V"]+word_dict[word]["E"]+1) / (count_dict["B"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_BX = P_BX - math.log((word_dict[word]["A"]+word_dict[word]["V"]+word_dict[word]["E"]+1) / (count_dict["A"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_VX = P_VX - math.log((word_dict[word]["A"]+word_dict[word]["B"]+word_dict[word]["E"]+1) / (count_dict["A"]+count_dict["B"]+count_dict["E"]+len(word_dict)))
                P_EX = P_EX - math.log((word_dict[word]["A"]+word_dict[word]["B"]+word_dict[word]["V"]+1) / (count_dict["A"]+count_dict["B"]+count_dict["V"]+len(word_dict)))

        probs = [P_AX, P_BX, P_EX, P_VX]
        outcome[i] = target[probs.index(max(probs))]
    return outcome

def evaluate(result, y):
    score = len(y)
    for i in range(len(y)):
        if result[i] != y[i]:
            score -= 1
    return score/len(y)

# Evaluate
ind = round(len(X)*0.7)
X_train = X[:ind]
X_val = X[ind:]
y_train = y[:ind]
y_val = y[ind:]

'''
prior_dict, word_dict, count_dict = train(X_train, y_train)
result = predict(X_val, prior_dict, word_dict, count_dict)
evaluate(result, y_val)
'''
prior_dict, word_dict, count_dict = train(X, y)
result = predict(X_test, prior_dict, word_dict, count_dict)
idh = [i+1 for i in range(1000)]
np.savetxt("shon866_A3_final_new.csv", np.c_[idh,result],fmt='%s',header="id,class", comments='', delimiter=',')

In [ ]:
# Complement class + IDF version
def train(X,y):
    
    # Obtain number of each class
    count_a = 0
    count_b = 0
    count_v = 0
    count_e = 0
    unique = {}
    word_dict = {}
    # Obtain number of words in each class
    count_dict = {"A":0, "B":0, "E":0, "V":0}

    for i in range(len(X)):
        word_li = X[i].split()
        for word in word_li:
            if word not in word_dict:
                word_dict[word]={"A":0, "B":0, "V":0, "E":0}
            if y[i] == "A":
                word_dict[word]["A"] += 1
                count_dict["A"] += 1
            elif y[i] == "B":
                word_dict[word]["B"] += 1
                count_dict["B"] += 1
            elif y[i] == "E":
                word_dict[word]["E"] += 1
                count_dict["E"] += 1
            elif y[i] == "V":
                word_dict[word]["V"] += 1
                count_dict["V"] += 1
            if word not in unique:
                    unique[word] = np.zeros((4000,), dtype=int)
                    unique[word][i] = 1
            else:
                if unique[word][i] == 0:
                    unique[word][i] = 1
                
    # Obtain Prior Probability of Classes
    for i in range(len(X)):
        if y[i] == "A":
            count_a += 1
        elif y[i] == "B":
            count_b += 1
        elif y[i] == "E":
            count_e += 1  
        elif y[i] == "V":
            count_v += 1
            
    prior_dict = {}
    prior_dict["A"] = math.log(count_a / len(X))
    prior_dict["B"] = math.log(count_b / len(X))
    prior_dict["V"] = math.log(count_v / len(X))
    prior_dict["E"] = math.log(count_e / len(X))            
    
    return prior_dict, word_dict, count_dict, unique

def predict(X, prior_dict, word_dict, count_dict, unique):
    # Predicting outcomes 
    outcome = [[] for x in range(len(X))]
    target = ["A", "B", "E", "V"]
    for i in range(len(X)):
        unseen = 0
        P_AX = prior_dict["A"]
        P_BX = prior_dict["B"]
        P_EX = prior_dict["E"]
        P_VX = prior_dict["V"]
        for word in X[i].split():
            if word not in word_dict:
                P_AX = P_AX - math.log(1/(count_dict["B"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_BX = P_BX - math.log(1/(count_dict["A"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_VX = P_VX - math.log(1/(count_dict["A"]+count_dict["B"]+count_dict["E"]+len(word_dict)))
                P_EX = P_EX - math.log(1/(count_dict["A"]+count_dict["B"]+count_dict["V"]+len(word_dict)))
            else:
                P_AX = P_AX - math.log(((word_dict[word]["B"]+word_dict[word]["V"]+word_dict[word]["E"])*math.log((4000/unique[word].sum()))+1) / (count_dict["B"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_BX = P_BX - math.log(((word_dict[word]["A"]+word_dict[word]["V"]+word_dict[word]["E"])*math.log((4000/unique[word].sum()))+1) / (count_dict["A"]+count_dict["V"]+count_dict["E"]+len(word_dict)))
                P_VX = P_VX - math.log(((word_dict[word]["A"]+word_dict[word]["B"]+word_dict[word]["E"])*math.log((4000/unique[word].sum()))+1) / (count_dict["A"]+count_dict["B"]+count_dict["E"]+len(word_dict)))
                P_EX = P_EX - math.log(((word_dict[word]["A"]+word_dict[word]["B"]+word_dict[word]["V"])*math.log((4000/unique[word].sum()))+1) / (count_dict["A"]+count_dict["B"]+count_dict["V"]+len(word_dict)))

        probs = [P_AX, P_BX, P_EX, P_VX]
        outcome[i] = target[probs.index(max(probs))]
    return outcome

def evaluate(result, y):
    score = len(y)
    for i in range(len(y)):
        if result[i] != y[i]:
            score -= 1
    return score/len(y)

# Evaluate
ind = round(len(X)*0.7)
X_train = X[:ind]
X_val = X[ind:]
y_train = y[:ind]
y_val = y[ind:]

prior_dict, word_dict, count_dict, unique = train(X_train, y_train)
result = predict(X_val, prior_dict, word_dict, count_dict, unique)
print(evaluate(result, y_val))

#---------------------------------------------------------------------------------------------------
#prior_dict, word_dict, count_dict, unique = train(X, y)
#result = predict(X_test, prior_dict, word_dict, count_dict, unique) #TRAINING ON TEST SET NOW !!!!!
#idh = [i+1 for i in range(1000)]
#np.savetxt("shon866_A3_final.csv", np.c_[idh,result],fmt='%s',header="id,class", comments='', delimiter=',')
